<a href="https://colab.research.google.com/github/N-aksif-N/MineColab_Improved/blob/free-config/MineColabImproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>
  <img style="display: block; margin-left: auto; margin-right: auto;" src="https://raw.githubusercontent.com/N-aksif-N/MineColab/master/minecolab.png" alt="" width="170" height="136" />
</p>

<h1 style="text-align: center;">
  <span style="color: #00ffff;">MineColab</span>
</h1>

<hr />
  <h2 style="text-align: center;">
    <span style="color: #99ccff;">Forked By N-Aksif</span>
  </h2>

  <h2 style="text-align: center;">
    <span style="color: #FFFFFF;">Run Minecraft Server On Google Colab</span>
  </h2>
  </span>
  </h2>
  <a href="https://github.com/N-aksif-N/MineColab/tree/main" target="_parent"><img src="https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white" align="right" alt="Open In Github"></a>
  <!-- Badges: https://ileriayo.github.io/markdown-badges/ -->

>[🔥 Starting](#scrollTo=Ei7_3ODcUdm_)

>[▶ or 🛑  Console](#scrollTo=JMgdsuPcUuEv)

>[⚓ Options](#scrollTo=HD4lBUT9RcS-)

>[📎  Log](#scrollTo=hfUxmQTgUF8Z)

>[📰  Software](#scrollTo=JkGq3KakW-Bc)

>[🎈  Plugins, mods](#scrollTo=QaNRBRuK8seq)

>[📁 File Management](#scrollTo=F7efc3oOqTVQ)



----

In [ ]:
# @markdown ##**[❗]  Set up** {display-mode: "form"}

# @markdown Check out [wiki of this project](https://github.com/N-aksif-N/MineColab_Improved/wiki) for more explanations
from requests import get
from time import sleep
from json import load, dump
from os.path import exists
from os import makedirs
from IPython.display import clear_output
%pip install -q jproperties
%pip install -q rich
from rich import print
%pip install -q pyngrok
%pip install -q BeautifulSoup4
%pip install -q pyyaml
if exists('/content/drive') == False:
  from google.colab import drive
  drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/minecraft'
#------------------------------------------------------------------------------------------------------------------------------------#
def LOG(*args, sep=''):
  check = False
  args = list(args)
  for i in range(len(args)):
    args[i] = str(args[i])
    if '\n' in args[i]: args[i] = args[i].replace('\n', ''); args.insert(0, '\n[bold green][ LOG ][/bold green] '); check = True; break
  if check == False: args.insert(0, '[bold green][ LOG ][/bold green] ')
  print(sep.join(map(str, args)))
def ERROR(*args, sep=''):
  %cd $drive_path
  clear_output()
  check = False
  args = list(args)
  for i in range(len(args)):
    args[i] = str(args[i])
    if '\n' in args[i]: args[i] = args[i].replace('\n', ''); args.insert(0, '\n[ ERROR ] '); check = True; break
  if check == False: args.insert(0, '[ ERROR ] ')
  raise Exception(sep.join(map(str, args)))
def MKDIR(path):
  try:
    makedirs(path, exist_ok = True)
    LOG(f'Directory {path} created')
  except: ERROR(f'Directory {path} already existed')

def GET(url):
  r = get(url)
  # Check gate
  if r.status_code == 200:
    return r
  else: ERROR('Error '+ str(r.status_code) + "! Most likely you entered an unsupported version. Try running the code again if you think that shouldn't have happened.")
def DOWNLOAD_FILE(url, path, file_name):
  # Check gate
  if exists( path+ '/' + file_name): LOG(f'File {file_name} already existed')
  else:
    # Download file into file_name thourgh url
    r = GET(url)
    LOG('\nDownloading ' + file_name)
    with open(path + '/' + file_name, 'wb') as f:
      f.write(r.content)

def COLABCONFIG(server_name):
  return f"{drive_path}/{server_name}/colabconfig.txt"
def COLABCONFIG_LOAD(server_name):
  if exists(COLABCONFIG(server_name)): return load(open(COLABCONFIG(server_name)))
  else:
    if exists(COLABCONFIG(server_name)): dump({"server_type": "generic"}, open(COLABCONFIG(server_name), 'w')) # using default, if config doesn't exist.
    ERROR('Please checking whether you deleted your colabconfig file or not.')

SERVERCONFIG = f'{drive_path}/server_list.txt'
def SERVER_IN_USE(server_name):
  if exists(f'{drive_path}/{server_name}') and server_name != '': return server_name
  else:
    serverconfig = load(open(SERVERCONFIG))
    if serverconfig['server_in_use'] != '': return serverconfig['server_in_use']
    else: ERROR('Please create a minecraft server first!')
# The jar file name
def JAR_LIST_RUN(server_version):
  return {'generic': 'server.jar', 'vanilla':'server.jar','snapshot': 'server.jar',   # NORMAL
          'purpur' : 'server.jar', 'paper': 'server.jar', 'velocity' : 'server.jar',  # PLUGINS
          'fabric' : 'server.jar', # forge doesn't include in this category           # MODS
          'arclight' : 'server.jar', 'mohist': 'server.jar', 'banner': 'server.jar'}  # HYBRID

#------------------------------------------------------------------------------------------------------------------------------------#

!sudo apt update &>/dev/null &&echo 'apt cache successfully updated' || echo "apt cache update failed, you might receive stale packages"
!sudo apt upgrade &>/dev/null &&echo 'apt cache successfully upgraded' || echo "apt cache upgrade failed, you might receive stale packages"
!export DISPLAY=:0.0
if exists(drive_path) == False:
  !mkdir -p $drive_path &>/dev/null&
  sleep(40)
if exists(SERVERCONFIG) == False:
  dump({"server_list": [], "server_in_use": "", "ngrok_proxy" : {'authtoken' : '', "region" : ''}, "zrok_proxy": {"authtoken": ''}, 'localtonet_proxy': {"authtoken": ''}}, open(SERVERCONFIG, 'w'))
%cd $drive_path
LOG('Completed')

In [ ]:
# @markdown ####**Choosing minecraft server**
# @markdown ##### Helping to set the defualt server_name.
serverconfig = load(open(SERVERCONFIG))
if serverconfig['server_list'] == []: ERROR(' Creating your minecraft server before choosing')
else:
  LOG('\nAivailable server:\n')
  for server in serverconfig['server_list']: colabconfig = COLABCONFIG_LOAD(server); print(f'{server} : {colabconfig["server_type"]} - {colabconfig["server_version"]}')
  server = input('Your minecraft server: ')
  if server in serverconfig['server_list']: serverconfig['server_in_use'] = server
  else:
    ERROR('You choose the wrong server choose again')
    LOG('\nAivailable server:\n')
    for server in serverconfig['server_list']: colabconfig = COLABCONFIG_LOAD(server); print(f'{server} : {colabconfig["server_type"]} - {colabconfig["server_version"]}')
    while server not in serverconfig['server_list']:
      print('Type "None" to exit the console.')
      server = input('Your minecraft server: ')
      serverconfig['server_in_use'] = server
  dump(serverconfig, open(SERVERCONFIG, 'w'))
  LOG('Completed')

---

# 🔥 **Starting**
---


In [ ]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from os.path import exists
from json import load, dump

# @markdown ####**Create your server file**
server_name = "Minecraft-server" # @param {type:"string"}
server_type = "vanilla" # default server type -- Please don't change this !!
# Change to empty strings to get the full list of avalable version
# Remember to set the versions variable in install server function to 'vanilla - latest_version'
# if you wanna to install the latest server.jar
version = 'vanilla - latest_version' # Defualt: = 'vanilla - latest_version'
# @markdown ####Standard turnel service ( Aivailable turnel: [ngrok](https://ngrok.com/), [cloudfare-argo](https://www.cloudflare.com/) or [zrok.io](https://zrok.io/) or [playit.gg](https://playit.gg/) and [localtonet](https://localtonet.com))
tunnel_service = 'ngrok' #@param ['ngrok', 'argo', 'zrok', 'playit', 'localtonet']

#---------------------------------------------------------------------------------- SNALL FUNCTION ------------------------------------------------------------------------------------#

def SERVERSJAR(server_type, version, all = False, jar = False):
  #Get the download URL (jar) AND return the detailed versions for each software (all)
  if all:
    Server_Jars_All = {
      'paper': 'https://api.papermc.io/v2/projects/paper', 'velocity': 'https://api.papermc.io/v2/projects/velocity',
      'purpur': 'https://api.purpurmc.org/v2/purpur',
      'mohist': 'https://mohistmc.com/api/v2/projects/mohist', 'banner': 'https://mohistmc.com/api/v2/projects/banner'
    }
    if server_type == 'vanilla' or server_type=='snapshot':
      rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
      if server_type == 'vanilla': server_type = 'release'
      if version != 'vanilla - latest_version': server_version = [hit["id"] for hit in rJSON["versions"] if hit["type"] == server_type]
      else:
        return rJSON['latest']['release']

    elif server_type == 'paper' or  server_type == 'velocity' or server_type == 'purpur' or server_type == 'mohist' or server_type == 'banner':
      rJSON = GET(Server_Jars_All[server_type]).json()
      server_version = [hit for hit in rJSON["versions"]]

    elif server_type == 'fabric':
      rJSON = GET('https://meta.fabricmc.net/v2/versions/game').json()
      server_version = [hit['version'] for hit in rJSON if hit['stable'] == True]

    elif server_type == 'forge':
      rJSON = GET('https://files.minecraftforge.net/net/minecraftforge/forge/index.html')
      soup = BeautifulSoup(rJSON.content, "html.parser")
      server_version = [tag.text for tag in soup.find_all('a') if '.' in tag.text and '\n' not in tag.text]

    else:
      LOG('Before going deeper, please check out https://github.com/IzzelAliz/Arclight')
      rJSON = GET('https://files.hypoglycemia.icu/v1/files/arclight/minecraft').json()['files']
      server_version  = [hit['name'] for hit in rJSON]
    return server_version

  elif jar == True and version != None:
    # RETURN DOWNLOAD URL
    if server_type == 'vanilla' or server_type=='snapshot':
      rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
      if server_type == 'vanilla': server_type = 'release'
      for hit in rJSON["versions"]:
        if hit["type"] == server_type and hit['id'] == version:
          return GET(hit['url']).json()["downloads"]['server']['url']

    elif server_type == 'paper' or  server_type == 'velocity':
      build = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}').json()["builds"][-1]
      jar_name = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}').json()["downloads"]["application"]["name"]
      return f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}/downloads/{jar_name}'

    elif server_type == 'purpur':
      build = GET(f'https://api.purpurmc.org/v2/purpur/{version}').json()["builds"]["latest"]
      return f'https://api.purpurmc.org/v2/purpur/{version}/{build}/download'

    elif server_type == 'mohist' or server_type == 'banner':
      return GET(f'https://mohistmc.com/api/v2/projects/{server_type}/{version}/builds').json()["builds"][-1]["url"]

    elif server_type == 'fabric':
      installerVersion = GET('https://meta.fabricmc.net/v2/versions/installer').json()[0]["version"]
      fabricVersion = GET(f'https://meta.fabricmc.net/v2/versions/loader/{version}').json()[0]["loader"]["version"]
      return "https://meta.fabricmc.net/v2/versions/loader/" + version + "/" + fabricVersion + "/" + installerVersion + "/server/jar"

    elif server_type == 'forge':
      rJSON = GET(f'https://files.minecraftforge.net/net/minecraftforge/forge/index_{version}.html')
      soup = BeautifulSoup(rJSON.content, "html.parser")
      tag =  soup.find('a', title="Installer"); tag = str(tag); tag = tag[tag.find('"') + 1 :]
      link = tag[:tag.find('"')]; link = link[link.find('=') + 1:]; link = link[link.find('=') + 1:]
      return link

    else:
      rJSON = GET(f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders').json()
      LOG('Available type: '); print([hit['name'] for hit in rJSON['files']])
      build = input(' Type: '); choice = input('Stable(st) or Snapshot(sn): ')
      if 'sn' in choice.lower(): choice = 'latest-snapshot';
      else: choice = "latest-stable";
      return f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders/{build}/{choice}'
  else: ERROR('Wrong given arguments')

#---------------------------------------------------------------------------------- MAIN CODE ------------------------------------------------------------------------------------#

# Auditing whether file is existed.
if exists(f'{drive_path}/{server_name}') == False:
  # Create folder
  LOG('Wait for 40 seconds')
  MKDIR(f'{drive_path}/{server_name}')
  sleep(40)

  # Get version
  if version == '':
    # Print available version => get version (string variable)
    LOG(f'Latest versions/builds for {server_type}')
    server_version = SERVERSJAR(server_type, version, all = True)
    print('\n Available version/builds:\n', server_version)
    version = input('Server versions: ')
    while version == '':
      LOG("Invalid versions")
      print('\n Available version/builds:\n', server_version)
      version = input('Server version: ')
      version = version
  elif version == 'vanilla - latest_version': version = SERVERSJAR(server_type, version, all = True)

  # Load serverconfig
  serverconfig = load(open(SERVERCONFIG))
  serverconfig['server_list'] += [server_name]
  serverconfig['server_in_use'] = server_name
  if serverconfig['ngrok_proxy'] == {"authtoken": "", "region": ""} and tunnel_service == 'ngrok':
    LOG('Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
    serverconfig['ngrok_proxy']['authtoken'] = input('Your authtoken: ')
    LOG('Available Regions:', ' ap - Asia/Pacific (Singapore)', ' au - Australia (Sydney)', ' eu - Europa (Frankfurt - Germany)', ' in - India (Mumbai)', ' jp - Japan (Tokyo)', ' sa - America (São Paulo - Brazil)', ' us - United States (Ohio)', sep='\n')
    serverconfig['ngrok_proxy']['region'] = input('Region: ')
  elif tunnel_service == 'zrok' and serverconfig['zrok_proxy'] == {'authtoken': ''}:
    # Settings variable
    serverconfig['zrok_proxy']['authtoken'] = input('Your zrok token: ')
  elif tunnel_service == 'localtonet' and serverconfig['localtonet_proxy'] == {'authtoken': ''}: LOG('Get your authtoken from https://localtonet.com/usertoken'); serverconfig['localtonet_proxy']['authtoken'] = input('Your localtonet token: ')
  dump(serverconfig, open(SERVERCONFIG, 'w'))

  # Set up colabconfig
  colabconfig = {"server_type": server_type, "server_version": version, "tunnel_service" : tunnel_service}
  dump(colabconfig, open(COLABCONFIG(server_name),'w'))

  # Download jar file
  if server_type == 'forge': jarname = 'forge-installer.jar' # The jar file name (forge need a special process)
  else: jarname = JAR_LIST_RUN(version)[server_type]
  DOWNLOAD_FILE(url= SERVERSJAR(server_type, version, jar = True), path = f"{drive_path}/{server_name}", file_name= jarname)
  sleep(40)
  LOG('\nCompleted!')
else: ERROR('Lol, you have already installed this server file')

In [ ]:
from os.path import exists
from time import sleep
from json import load, dump
# @markdown ####**Delete server**
server_name = '' #  Get default server

serverconfig = load(open(SERVERCONFIG)); server_name = serverconfig['server_in_use']
if serverconfig['server_list'] == []: ERROR("You haven't installed yet.")
else:

  # Auditing whether file is existed.
  if exists(f'{drive_path}/{server_name}') == False: ERROR("You haven't installed yet.")
  LOG(f'Deleting {server_name}...')

  # Delete folder without noticable
  !rm -rf "{drive_path}/{server_name}" >/dev/null

  # Remove the folder name in server config txt files
  serverconfig['server_list'].remove(server_name)
  if serverconfig['server_in_use'] == server_name:
    try: serverconfig['server_in_use']= serverconfig['server_list'][0]
    except: serverconfig['server_in_use'] = ''
  dump(serverconfig, open(SERVERCONFIG, 'w'))

  LOG('Wait for a while.')
  sleep(40)
  LOG('Completed')

-----------------------------------------------------

# ▶ **or** 🛑  **Console**
---
The main console for your minecraft server

In [ ]:
# @markdown
from os import pathsep, environ
from os.path import exists, join, expanduser
from pyngrok import conf, ngrok
from IPython.display import clear_output
from time import sleep
from json import loads
import yaml

# GUIDANCE
choice = input('show(s)/hind(h) the notice part? - ')
if 's' in choice: clear_output(); print("\n\n- **[Ngrok](https://ngrok.com)**\n  + Follow the prompts.\n  + The IP will change whenever you restart the server.\n\n- **[Cloudflare's argo](https://www.cloudflare.com/)** :\n    - If the 'Your free tunnel has started!' notification appears => Done.\n    - Access to your server:\n    1. Download [Cloudflared client](https://github.com/cloudflare/cloudflared/releases/).\n    2. Launch the binary with `<your Cloudflare file name> access tcp --hostname <tunnel_address> --url 127.0.0.1:25565` (note: tunnel_address is your address which has been set on your Cloudflare).\n    4. Finally, connect to `127.0.0.1:25565` from the minecraft client which is located in that machine.\n\n- **[Localtonet](https://localtonet.com/)**:\n  1. Navigate to [TCP-UDG](https://localtonet.com/tunnel/tcpudp) page. Select TCP in Protocol Types.\n  2. Get your authtoken from [Authtoken](https://localtonet.com/usertoken).\n  3. Pick the server you'd like your tunnel to operate on.\n  4. Input the IP and Port values the tunnel will listen to, in this case, for Minecraft, it's typically IP: 127.0.0.1 and Port: 25565.\n  5. Finally, create and start your tunnel by pressing the Start button.\n  - Read more on [how-to-use-localtonet-with-minecraft](https://localtonet.com/documents/using-localtonet-with-minecraft)\n\n- **[Zrok](https://zrok.io/)**:\n  1. Download the zrok app through [link](https://docs.zrok.io/docs/getting-started/)\n  2. Open the shell. Type `zrok invite` to sign up and get the authtoken\n  3. Follow the prompts\n\n- **[PlayIt](https://playit.gg/)**: follow the prompts."); sleep(20)
clear_output()

# Variables
server_name = ''
server_name = SERVER_IN_USE(server_name)
colabconfig = COLABCONFIG_LOAD(server_name)
serverconfig = load(open(SERVERCONFIG))
_type = colabconfig["server_type"]
version = colabconfig["server_version"]
tunnel_service = colabconfig["tunnel_service"]

#---------------------------------------------------------------------------------- SNALL FUNCTION ------------------------------------------------------------------------------------#

def CONNECT_NGROK(port, type_, proxy):
    # Get serverconfig['ngrok_proxy'] : dict includes (authtoken, region)
    token = proxy['authtoken']
    !ngrok authtoken $token
    region = proxy['region']
    conf.get_default().region = region
    url = ngrok.connect (port, type_)
    if type_ == 'tcp': return 'Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', '')
    else: return url
def CONFIG_ZROK(serverconfig):
    %cd $drive_path
    # Enable zrok
    result = ! zrok status
    if 'To create a local environment' in str(result): LOG('Status: \n'); print(result)
    else:
      # Creating zrok directory
      zrok_dir = "tunnel/zrok" ; zrok_tar = 'zrok_0.4.32_linux_amd64.tar.gz';
      if exists(f"{zrok_dir}") == False:
        MKDIR(zrok_dir)
        DOWNLOAD_FILE(url = 'https://github.com/openziti/zrok/releases/download/v0.4.32/zrok_0.4.32_linux_amd64.tar.gz', path = 'tunnel', file_name = zrok_tar)
      if exists(f"{zrok_dir}/{zrok_tar}") == False:
        ! sudo cp -r tunnel/$zrok_tar $zrok_dir/$zrok_tar
        ! tar -xf $zrok_dir/$zrok_tar -C $zrok_dir > /dev/null &&echo "Installing zrok done" || echo "Installing zrok failed"

      ! chmod +x $zrok_dir/zrok
      LOG('Status: \n')
      ! ./$zrok_dir/zrok version
      ! ./$zrok_dir/zrok status
      token = serverconfig['zrok_proxy']['authtoken']
      ! ./$zrok_dir/zrok enable $token && echo 'ZROK ENABLED' || echo 'ZROK NOT ENABLED. Try run the server again.'
def RUNCOMMAND(server_name, serverconfig, version, _type, tunnel_service, hide = False):
    %cd $drive_path
    # Creating directories:
    #      tunnel - include all the tunnel sevice
    #      logs - include the tunnel logs
    MKDIR('tunnel'); MKDIR('logs')
    # Get all the improving java arguments
    java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
    args = " -Xms8G -Xmx8G"
    if _type == "paper" or _type == 'purpur' or _type == 'arclight':
      # Improving paper cilent (purpur is an alternative).
      # For more detailed: https://docs.papermc.io/paper/aikars-flags
      args += ' -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true'
    elif _type == "velocity":
      # Java argument for velocity server
      # For more detailed: https://docs.papermc.io/velocity/tuning
      args += ' -XX:+UseG1GC -XX:G1HeapRegionSize=4M -XX:+UnlockExperimentalVMOptions -XX:+ParallelRefProcEnabled -XX:+AlwaysPreTouch -XX:MaxInlineLevel=15'
    # GC_LOGGING
    if java_ver[0] == "1": args += ' -Xloggc:gc.log -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps -XX:+UseGCLogFileRotation -XX:NumberOfGCLogFiles=5 -XX:GCLogFileSize=1M'
    # else: args += '-Xlog:gc*:logs/gc.log:time,uptime:filecount=5,filesize=1M' # GC_Logging may not worked in java 11 and above

    # Forge need to open and run it before starting server. => Using command instead.
    if _type == 'forge':
      !java -jar forge-installer.jar --installServer >/dev/null
    jar_name = ''
    if _type == 'forge':
      a = !ls -1 | grep .jar
      for i in a:
        if 'forge' in i and 'installer' not in i: jar_name = i
    else: jar_name = JAR_LIST_RUN(version)[_type]

    # Set up run server cmd
    cmd = ''
    if exists(f'{drive_path}/{server_name}/run.sh') == True and _type != 'arclight':
      with open(f'{drive_path}/{server_name}/run.sh', 'r') as f: cmd = f.read()
    if cmd != '': cmd = cmd[cmd.find('java'): ].replace('@user_jvm_args.txt', args); cmd = cmd.replace('"$@"', 'nogui "$@"')
    else: cmd = f'java -server {args} -jar {jar_name} nogui'

    if hide == True:
      if _type == 'forge':
        # Make run.sh running
        if exists(f'{drive_path}/{server_name}/eula.txt') == True: LOG('Run the server aigain to finished installing.')
      if _type != 'mohist' and _type != 'banner':
        # Install needed file: eula.txt and so on.
        ! $cmd > /dev/null

    else:
      # Starting tunneling and run java file.
      LOG(f'Starting server ({tunnel_service})...')
      LOG('Stop server => /stop')
      if tunnel_service == "ngrok":
        LOG(CONNECT_NGROK(type_= 'tcp', port= 25565, proxy = serverconfig['ngrok_proxy']))
        ! $cmd

      elif tunnel_service == "playit":
        # Download playit
        ! command -v playit || curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg > /dev/null && echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list && sudo apt -qq update && sudo apt install playit >/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
        # Run server
        LOG('Playit will run automatically in the background. To stop the playit service, run sudo systemctl stop playit ')
        ! playit setup
        ! playit &> logs/playit.txt & $cmd

      elif tunnel_service == "zrok":
        CONFIG_ZROK(serverconfig); LOG('Starting: ')
        ! $cmd & ./tunnel/zrok/zrok share public --backend-mode tcpTunnel 127.0.0.1:25565 --headless &> logs/zrok2.txt &

      elif tunnel_service == 'localtonet':
        if exists(f'{drive_path}/{server_name}/tunnel/localtonet') == False:
          MKDIR('localtonet'); DOWNLOAD_FILE(url = 'https://localtonet.com/download/localtonet-linux-x64.zip', path = 'tunnel/localtonet', file_name = 'localtonet-linux-x64.zip')
          ! sudo unzip localtonet-linux-x64.zip > /dev/null && echo 'Downloaded' || echo 'Failed to download'
          ! sudo chmod 777 +x ./tunnel/localtonet/localtonet
        LOG('Starting: '); token = serverconfig['localtonet_proxy']['authtoken']
        ! ./tunnel/localtonet/localtonet authtoken $token &> logs/localtonet.txt & $cmd

      elif tunnel_service == 'argo':
        # Download argo
        if exists(f'{drive_path}/{server_name}/tunnel/cloudflared-linux-amd64') == False:
          DOWNLOAD_FILE(url = 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64', path = 'tunnel', file_name= 'cloudflared-linux-amd64')
          ! chmod 777 cloudflared-linux-amd64
        # Run server
        ! ./tunnel/cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 && echo 'Your free tunnel has started!' && $cmd
def GEYSER_CONFIG(server_name, colabconfig, _type):
    if 'Geysermc' in colabconfig:
      if 'fabric' in _type: path = f'{drive_path}/{server_name}/config/Geyser-Fabric/config.yml'
      elif'paper' in _type or 'purpur' in _type: path = f'{drive_path}/{server_name}/plugins/Geyser-Spigot/config.yml'
      else:
        LOG('Check out https://wiki.geysermc.org/floodgate/setup/ for installing Floodgate on servers behind the proxy'); LOG('Note for Fabric behind a Velocity proxy: You will need to configure FabricProxyLite to allow the Fabric server to receive data from Velocity.'); LOG('After done settings, copy the key.pem file in the proxy Floodgate config folder to all backend servers’ Floodgate config folder.'); LOG('DO NOT DISTRIBUTE THIS KEY TO ANYBODY! This key is what allows for Bedrock accounts to bypass the Java Edition authentication, and if anyone gets ahold of this, they can wreak havoc on your server.')
        path =  f'{drive_path}/{server_name}/plugins/Geyser-Velocity/config.yml'
      # To get the requirement
      if exists(path):
        LOG('Config geyser plugin')
        # Configuring
        with open(path, 'r') as file: config = yaml.safe_load(file); config['bedrock']['clone-remote-port'] = 'true'; config['auth-type'] = 'floodgate'; yaml.dump(config, file)
        # Clear notification
        colabconfig.pop('Geysermc')
        dump(colabconfig, open(COLABCONFIG(server_name),'w'))
        LOG('Config done. Check out https://wiki.geysermc.org/geyser/playit-gg for setting tunnels \n - Verify whether connections from other networks are possible by running geyser connectiontest <ip>:<port> in the console.')
def SUPPORT_DYNMAP(colabconfig):
    if exists(f'{drive_path}/{server_name}/mods/dynmap-server.jar') or exists(f'{drive_path}/{server_name}/plugins/dynmap-server.jar'):
      if colabconfig['tunnel_service'] == 'ngrok': LOG('\n YOUR WEB_SERVER (dynmap): \n'); LOG(CONNECT_NGROK(type_= 'http', port= 8123, proxy = serverconfig['ngrok_proxy']))
      elif colabconfig['tunnel_service'] == 'playit': LOG('Access to https://playit.gg/support/add-dynmap-to-minecraft/ then head over to step 5. DO IT!!!!!!!')
      elif colabconfig['tunnel_service'] == 'localtonet': LOG('GUIDE: \n - Go to https://localtonet.com/tunnel/http \n - Select Process Type for your needs. (Random Sub Domain, Custom Sub Domain, Custom Domain)\n - Select your authtoken. Get it from https://localtonet.com/usertoken. And then select the Server you want your tunnel to run on.\n Enter the IP: 127.0.0.1 and Port: 8123 values that the tunnel will listen to\n - Create and Start your tunnel by pressing the Start Button from the list.')
      elif colabconfig['tunnel_service'] == 'zrok':
        !./tunnel/zrok/zrok share public localhost:8123 --headless &>logs/zrok.txt&
        sleep(50)
        %cd $drive_path
        with open('/content/drive/MyDrive/minecraft/logs/zrok.txt', 'r') as f:
          content = f.read()
          json_data = []
          while content.find('}') != -1:
            json = content[content.find('{') : content.find('}') + 1]
            content = content[content.find('}') + 1 :]
            json_data.append(loads(json))
        print('Your dynmap webserver: ', json_data[1]['msg'][json_data[1]['msg'].find('https://') :])
print('Your dynmap webserver: ', json_data[1]['msg'][json_data[1]['msg'].find('https://') :])

def INSTALL_JAVA(version):
    # Adoptopen jdk -- optional
    # !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
    # !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk gets installed."

    # Java 8 is required to run Minecraft versions 1.12 through 1.17. Java 17 is required to run Minecraft version 1.18 and up.
    if version >= '1.20.5':
      ! sudo apt-get install -y openjdk-21-jre-headless  > /dev/null && echo 'Openjdk21 is working correctly, you are good to go.' || "Openjdk21 doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.20.5 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64/bin/java"
      ! update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
    elif version >= "1.17":
      ! sudo apt-get -y install openjdk-17-jre-headless  > /dev/null && echo 'Openjdk17 is working correctly, you are good to go.' || "Openjdk17 doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
      ! update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
    else:
      ! sudo apt-get -y install openjdk-8-jre-headless   > /dev/null && echo 'Openjdk8 is working correctly, you are good to go.' || echo "Openjdk8 doesn't seem to be installed or isn't working, falling back to java 11\nYou might experience reduced performance. Minecraft 1.18 and above might fail to launch."
      environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
      ! update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
def EXIT_NGROK():
    # Exiting ngrok tunnel helps improve the performance
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels: ngrok.disconnect(tunnel.public_url)
    ngrok.kill()
    LOG("Ngrok closed.")

#---------------------------------------------------------------------------------- MAIN CODE ------------------------------------------------------------------------------------#

# Cd to server path
%cd $drive_path/$server_name
# Install java acording to the version of server.jar
INSTALL_JAVA(version)
# Install necessary files
if exists(f'{drive_path}/{server_name}/eula.txt') == False:
  LOG("\n[Note: Downloading essentials files takes approximately 10-15 minutes. Please wait for a little bit.]")
  RUNCOMMAND(server_name, serverconfig, version, _type, tunnel_service, hide = True)
with open(f'{drive_path}/{server_name}/eula.txt', 'w') as f:
  # Accept EULA
  LOG('\nAccepting Eula.')
  f.seek(0)
  f.write('eula=true')
# See what changes in the folder
LOG("\nFiles:")
!ls -a
# Run minecraft server
SUPPORT_DYNMAP(colabconfig)
RUNCOMMAND(server_name, serverconfig, version, _type, tunnel_service)
GEYSER_CONFIG(server_name, colabconfig, _type)
# Stop minecraft server => Exit tunnel => Cd to home
LOG("Finalized server.")
if colabconfig['tunnel_service'] == 'ngrok': EXIT_NGROK()
%cd $drive_path

---

# ⚓ **Options**
---


In [ ]:
from jproperties import Properties
from os.path import exists
from google.colab import files as fls

# @markdown ###**Server_custom**
choice = 'server_icon' # @param ["server_motd", "server_icon"]

# Default server
server_name = SERVER_IN_USE(server_name)
if exists(f"{drive_path}/{server_name}/server.properties") == False: ERROR(' Running your minecraft server before editing properties')
else:
  LOG('Server:  ' + server_name)
  # Add server icon file
  if choice == 'server_icon':
    if exists(f"{drive_path}/{server_name}/server-icon.png"): LOG('There is an existing server_icon.png file. If you wanna to change, entering'); input()
    # Upload image and get direct link from https://postimages.org/
    # or leave it emty. it will download the file.
    url = input('Download_url (be able to leave it emty) : ')
    if url != '': DOWNLOAD_FILE( url = url, path = f'{drive_path}/{server_name}', file_name = 'server-icon.png')
    else:
      uploaded = fls.upload()
      file_ = uploaded.keys()
      try:
        file_ = [fn for fn in uploaded.keys()][0]
        !sudo mv -f $drive_path/$file_ $drive_path/$server_name/'server-icon.png'
        sleep(40)
      except: ERROR("You didn't upload server-icon.png")
  else:
    LOG('Get MOTD from https://mctools.org/motd-creator')
    server_properties = Properties()
    with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
        server_properties.load(f, "utf-8")
    server_properties["motd"] = input('Your MOTD  :  ')
    with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
        server_properties.store(f, encoding="utf-8")
  LOG('Completed')

In [ ]:
from jproperties import Properties
from os.path import exists

# @markdown ###**Server properties**

Slots = 20 # @param {type:"slider", min:0, max:100, step:1}
Gamemode = "survival" # @param ["survival", "creative", "adventure", "spectator"]
Difficulty = "easy" # @param ["peaceful", "easy", "normal", "hard"]
Cracked = True # @param {type:"boolean"}
PVP = True # @param {type:"boolean"}
Command_block = True # @param {type:"boolean"}
Fly = True # @param {type:"boolean"}
Animals = True # @param {type:"boolean"}
Monsters = True # @param {type:"boolean"}
Villagers = True # @param {type:"boolean"}
Nether = True # @param {type:"boolean"}
Force_gamemode = False # @param {type:"boolean"}
Spawn_protection = 16 # @param {type:"slider", min:0, max:100, step:1}

#---------------------------------------------------------------------------------- MAIN CODE ------------------------------------------------------------------------------------#

server_name = SERVER_IN_USE(server_name)
if exists(f"{drive_path}/{server_name}/server.properties") == False: ERROR(' Running your minecraft server before editing properties')
else:
  server_properties = Properties() # Download file
  with open(f"{drive_path}/{server_name}/server.properties", "rb") as f:
      server_properties.load(f, "utf-8")
  # Configuring
  server_properties["max-players"] = str(Slots)
  server_properties["gamemode"] = Gamemode
  server_properties["difficulty"] = Difficulty
  dict_ = {'pvp': PVP, 'enable-command-block': Command_block, 'allow-flight': Fly, 'spawn-animals': Animals, 'spawn-monsters': Monsters,
          'spawn-npcs': Villagers, 'allow-nether': Nether, 'force-gamemode': Force_gamemode, 'spawn-protection': Spawn_protection, "online-mode" : Cracked}
  for keys, value in dict_.items(): server_properties[keys] = str(value).lower()
  # Saving
  with open(f"{drive_path}/{server_name}/server.properties", "wb") as f:
      server_properties.store(f, encoding="utf-8")
  LOG('Completed')


---

# 📎  **Log**
---


In [ ]:
from os.path import exists

# @markdown **Shows latest log of your minecraft server**
# Getting server_folder
server_name = SERVER_IN_USE(server_name)
# Check log
if exists(f'/content/drive/My Drive/minecraft/{server_name}/logs/latest.log'):
  LOG(f'Server {server_name}')
  # Open and get log content
  with open(f'/content/drive/My Drive/minecraft/{server_name}/logs/latest.log', 'r') as f: content = f.read(); print(content); LOG('\nTry access to https://mclo.gs to see problems and the way to deal with.')
else: ERROR("Error: File didn't exists")

---


# 📰  **Software**
---
Change the software (vanilla) to others platforms

---


+ Please checking whether your minecraft server is in Gdrive or not.


In [ ]:
from json import load, dump
from requests import get
from bs4 import BeautifulSoup
from os.path import exists
from time import sleep

# @markdown ####Choosing server type
server_type = "vanilla" # @param ["vanilla", "snapshot", "paper", "purpur", "forge", "fabric", 'velocity', 'arclight','mohist', 'banner']
# @markdown ####Standard turnel service ( Aivailable turnel: [ngrok](https://ngrok.com/), [cloudfare-argo](https://www.cloudflare.com/) or [zrok.io](https://zrok.io/) or [playit.gg](https://playit.gg/) and [localtonet](https://localtonet.com))
tunnel_service = 'ngrok' #@param ['ngrok', 'argo', 'zrok', 'playit', 'localtonet']

serverconfig = load(open(SERVERCONFIG)); colabconfig = load(open(COLABCONFIG(server_name)))
if server_name == '': server_name = serverconfig['server_in_use']

#---------------------------------------------------------------------------------- DELETE SERVER ------------------------------------------------------------------------------------#

if serverconfig['server_list'] == []: ERROR("You haven't installed yet.")
else:
  # Auditing whether file is existed.
  if exists(f'{drive_path}/{server_name}') == False: ERROR("You haven't installed yet.")
  LOG(f'Deleting {server_name}...')

  # Delete folder without noticable
  !rm -rf "{drive_path}/{server_name}" >/dev/null

  # Remove the folder name in server config txt files
  serverconfig['server_list'].remove(server_name)
  if serverconfig['server_in_use'] == server_name:
    try: serverconfig['server_in_use']= serverconfig['server_list'][0]
    except: serverconfig['server_in_use'] = ''
  dump(serverconfig, open(SERVERCONFIG, 'w'))

  LOG('Wait for a while.'); sleep(40); LOG('Completed')

#---------------------------------------------------------------------------------- SLEEPING ------------------------------------------------------------------------------------#

sleep(40); version = '';
if tunnel_service == 'default': tunnel_service = colabconfig['tunnel_service']

#---------------------------------------------------------------------------------- INSTALL SERVER ------------------------------------------------------------------------------------#

def SERVERSJAR(server_type, version, all = False, jar = False):
  #Get the download URL (jar) AND return the detailed versions for each software (all)
  if all:
    Server_Jars_All = {
      'paper': 'https://api.papermc.io/v2/projects/paper', 'velocity': 'https://api.papermc.io/v2/projects/velocity',
      'purpur': 'https://api.purpurmc.org/v2/purpur',
      'mohist': 'https://mohistmc.com/api/v2/projects/mohist', 'banner': 'https://mohistmc.com/api/v2/projects/banner'
    }
    if server_type == 'vanilla' or server_type=='snapshot':
      rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
      if server_type == 'vanilla': server_type = 'release'
      if version != 'vanilla - latest_version': server_version = [hit["id"] for hit in rJSON["versions"] if hit["type"] == server_type]
      else:
        return rJSON['latest']['release']

    elif server_type == 'paper' or  server_type == 'velocity' or server_type == 'purpur' or server_type == 'mohist' or server_type == 'banner':
      rJSON = GET(Server_Jars_All[server_type]).json()
      server_version = [hit for hit in rJSON["versions"]]

    elif server_type == 'fabric':
      rJSON = GET('https://meta.fabricmc.net/v2/versions/game').json()
      server_version = [hit['version'] for hit in rJSON if hit['stable'] == True]

    elif server_type == 'forge':
      rJSON = GET('https://files.minecraftforge.net/net/minecraftforge/forge/index.html')
      soup = BeautifulSoup(rJSON.content, "html.parser")
      server_version = [tag.text for tag in soup.find_all('a') if '.' in tag.text and '\n' not in tag.text]

    else:
      LOG('Before going deeper, please check out https://github.com/IzzelAliz/Arclight')
      rJSON = GET('https://files.hypoglycemia.icu/v1/files/arclight/minecraft').json()['files']
      server_version  = [hit['name'] for hit in rJSON]
    return server_version

  elif jar == True and version != None:
    # RETURN DOWNLOAD URL
    if server_type == 'vanilla' or server_type=='snapshot':
      rJSON = GET('https://launchermeta.mojang.com/mc/game/version_manifest.json').json()
      if server_type == 'vanilla': server_type = 'release'
      for hit in rJSON["versions"]:
        if hit["type"] == server_type and hit['id'] == version:
          return GET(hit['url']).json()["downloads"]['server']['url']

    elif server_type == 'paper' or  server_type == 'velocity':
      build = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}').json()["builds"][-1]
      jar_name = GET(f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}').json()["downloads"]["application"]["name"]
      return f'https://api.papermc.io/v2/projects/{server_type}/versions/{version}/builds/{build}/downloads/{jar_name}'

    elif server_type == 'purpur':
      build = GET(f'https://api.purpurmc.org/v2/purpur/{version}').json()["builds"]["latest"]
      return f'https://api.purpurmc.org/v2/purpur/{version}/{build}/download'

    elif server_type == 'mohist' or server_type == 'banner':
      return GET(f'https://mohistmc.com/api/v2/projects/{server_type}/{version}/builds').json()["builds"][-1]["url"]

    elif server_type == 'fabric':
      installerVersion = GET('https://meta.fabricmc.net/v2/versions/installer').json()[0]["version"]
      fabricVersion = GET(f'https://meta.fabricmc.net/v2/versions/loader/{version}').json()[0]["loader"]["version"]
      return "https://meta.fabricmc.net/v2/versions/loader/" + version + "/" + fabricVersion + "/" + installerVersion + "/server/jar"

    elif server_type == 'forge':
      rJSON = GET(f'https://files.minecraftforge.net/net/minecraftforge/forge/index_{version}.html')
      soup = BeautifulSoup(rJSON.content, "html.parser")
      tag =  soup.find('a', title="Installer"); tag = str(tag); tag = tag[tag.find('"') + 1 :]
      link = tag[:tag.find('"')]; link = link[link.find('=') + 1:]; link = link[link.find('=') + 1:]
      return link

    else:
      rJSON = GET(f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders').json()
      LOG('Available type: '); print([hit['name'] for hit in rJSON['files']])
      build = input(' Type: '); choice = input('Stable(st) or Snapshot(sn): ')
      if 'sn' in choice.lower(): choice = 'latest-snapshot';
      else: choice = "latest-stable";
      return f'https://files.hypoglycemia.icu/v1/files/arclight/minecraft/{version}/loaders/{build}/{choice}'
  else: ERROR('Wrong given arguments')

#---------------------------------------------------------------------------------- MAIN CODE ------------------------------------------------------------------------------------#
# Auditing whether file is existed.
if exists(f'{drive_path}/{server_name}') == False:
  # Create folder
  LOG('Wait for 40 seconds')
  MKDIR(f'{drive_path}/{server_name}')
  sleep(40)

  # Get version
  if version == '':
    # Print available version => get version (string variable)
    LOG(f'Latest versions/builds for {server_type}')
    server_version = SERVERSJAR(server_type, version, all = True)
    print('\n Available version/builds:\n', server_version)
    version = input('Server versions: ')
    while version == '':
      LOG("Invalid versions")
      print('\n Available version/builds:\n', server_version)
      version = input('Server version: ')
      version = version
  elif version == 'vanilla - latest_version': version = SERVERSJAR(server_type, version, all = True)

  # Load serverconfig
  serverconfig = load(open(SERVERCONFIG))
  serverconfig['server_list'] += [server_name]
  serverconfig['server_in_use'] = server_name
  if serverconfig['ngrok_proxy'] == {"authtoken": "", "region": ""} and tunnel_service == 'ngrok':
    LOG('Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
    serverconfig['ngrok_proxy']['authtoken'] = input('Your authtoken: ')
    LOG('Available Regions:', ' ap - Asia/Pacific (Singapore)', ' au - Australia (Sydney)', ' eu - Europa (Frankfurt - Germany)', ' in - India (Mumbai)', ' jp - Japan (Tokyo)', ' sa - America (São Paulo - Brazil)', ' us - United States (Ohio)', sep='\n')
    serverconfig['ngrok_proxy']['region'] = input('Region: ')
  elif tunnel_service == 'zrok' and serverconfig['zrok_proxy'] == {'authtoken': ''}:
    # Settings variable
    serverconfig['zrok_proxy']['authtoken'] = input('Your zrok token: ')
  elif tunnel_service == 'localtonet' and serverconfig['localtonet_proxy'] == {'authtoken': ''}: LOG('Get your authtoken from https://localtonet.com/usertoken'); serverconfig['localtonet_proxy']['authtoken'] = input('Your localtonet token: ')
  dump(serverconfig, open(SERVERCONFIG, 'w'))

  # Set up colabconfig
  colabconfig = {"server_type": server_type, "server_version": version, "tunnel_service" : tunnel_service}
  dump(colabconfig, open(COLABCONFIG(server_name),'w'))

  # Download jar file
  if server_type == 'forge': jarname = 'forge-installer.jar' # The jar file name (forge need a special process)
  else: jarname = JAR_LIST_RUN(version)[server_type]
  DOWNLOAD_FILE(url= SERVERSJAR(server_type, version, jar = True), path = f"{drive_path}/{server_name}", file_name= jarname)
  sleep(40)
  LOG('\nCompleted!')
else: ERROR('Lol, you have already installed this server file')

---

# 🎈  **Plugins, mods**

####Download modpack/mod/plugin from [curseforge](https://www.curseforge.com/Minecraft) and [modrinth](https://modrinth.com/)
---


In [ ]:
from pyngrok import conf, ngrok
from os import environ, pathsep
from zipfile import ZipFile
from requests import get, post
from time import sleep
from json import load, dump, loads
from google.colab import files as fls

# @markdown ###**Your download choice**
choice = 'search' #@param ['search', 'url', 'upload_file', 'install_geysermc', 'dynmap support']

#@markdown ### **Your search name**
#@markdown #####If your choice is url don't type it on the search name here.
search_name = '' # @param {type: 'string'}

# @markdown ####Choose the place to download mod/modpack/plugin
software = 'Curseforge' # @param ["Curseforge", "Modrinth"]

# @markdown ###**Details:**
# @markdown
# @markdown #### (none -> don't search (version -- doesn't wellcoming to do this), default -> set up according to colabconfig file)
categories = 'default'   #@param ['none', 'default', 'vanilla', 'fabric', 'forge', 'paper', 'purpur']
versions = "default" # @param ["default"] {allow-input: true}
project_types = 'default' # @param ['none', 'default', 'mods', 'plugins', 'modpacks']
index = 'none' #@param ['none', 'relevance', 'downloads', 'follows', 'newest', 'updated']

# CHecking and setting variable
server_name = SERVER_IN_USE(server_name)
if exists(f"{drive_path}/{server_name}/server.properties") == False: ERROR(' Running your minecraft server before editing properties')
else:
  colabconfig = COLABCONFIG_LOAD(server_name)
  if versions == 'default': versions = colabconfig["server_version"]
  if categories == 'default': categories = colabconfig['server_type']
  if project_types == 'default':
    if 'fabric' in categories or 'forge' in categories : project_types = 'mods'
    elif 'paper' in categories or 'purpur' in categories : project_types = 'plugins'
    elif 'arclight' in categories or 'mohist'  in categories or 'banner': project_types = input('Project type (mods or plugins or modpacks) : ');
  else: project_types = 'tmp';
  if 'arclight' in categories or 'mohist'  in categories or 'banner' in categories or 'vanilla' in categories or 'snapshot' in categories: categories = 'none';
  elif 'paper' in categories or 'purpur' in categories:
    if project_types == 'mods': ERROR(f'Hmm, maybe you install the wrong types. Your server_type (currently) is {categories}');
  elif 'fabric' in categories or 'forge' in categories:
    if project_types == 'plugins': ERROR(f'Hmm, maybe you install the wrong types. Your server_type (currently) is {categories}');
  if 'vanilla' in categories or 'snapshot' in categories:
    if project_types == 'modpacks': ERROR(f'Hmm, maybe you install the wrong types. Your server_type (currently) is {categories}');
  path = f'{drive_path}/{server_name}/{project_types}'
  if exists(path) == False:
    MKDIR(path)
    sleep(40)
  choice = choice
  if choice == 'url':
    if 'http' in server_name: LOG('HTTP is not safe web. Try using https instead.')
    if 'http' in server_name or 'https' in server_name: url = input('Url: ')

# I will not do remove the class file. Because:
#   1. It is hard and very slow to extract them all
#   2. It will broken in some specific runtime

Download_(choice = choice, url = url, server_name = server_name, categories = categories, versions= versions, proj_types= project_types, index= index).Install_( search_name = search_name.lower(), software = software)

class Download_:
  def __init__(self, choice, server_name, categories, versions, proj_types, index, url):
    self.server_name = SERVER_IN_USE(server_name)
    if exists(f"{drive_path}/{self.server_name}/server.properties") == False: ERROR(' Running your minecraft server before editing properties')
    else:
      self.colabconfig = COLABCONFIG_LOAD(self.server_name)
      self.categories = categories
      self.versions = versions
      self.proj_types = proj_types
      self.index = index
      self.path = f'{drive_path}/{self.server_name}/{self.proj_types}'
      self.choice = choice
      self.url = url
  def FACETS(self, software):
    # Get all the syntax
    categories = ''; index = ''; versions = ''
    if software == 'Modrinth':
      facets = "["
      if self.categories != 'none': facets += '["categories:' + self.categories + '"]';
      if facets != '[' and self.versions != 'none': facets += "," + '["versions:' + self.versions +'"]'
      elif self.versions != 'none': facets += '["versions:' + self.versions + '"]';
      if self.proj_types == 'mods': proj_types = 'mod'
      elif self.proj_types == 'plugins': proj_types = 'plugin'
      if facets != '[' and self.proj_types != 'tmp': facets += "," + '["project_type:' + proj_types + '"]'
      elif self.proj_types != 'tmp': facets += '["project_type:' + proj_types + '"]'
      facets += "]"; facetsInURL = "";
      if facets != "[]": facetsInURL += f'&facets={facets}'
      if self.index != "none": facetsInURL += f'&index={self.index}'
      return facetsInURL
    else:
      if self.categories != 'none':
        if self.categories == 'fabric': categories = 4 #Fabric
        elif self.categories == 'forge': categories = 1 #Forge
        elif self.categories == 'quilt': categories = 5 # quilt
        categories = f"&modLoaderType={categories}"
      if self.index != "none":
        if self.index == "relevance": index = 1 # Featured
        elif self.index == "downloads": index = 6 #TotalDownloads
        elif self.index == "follows": index = 2 #Popularity
        elif self.index == "newest": index = 11 #ReleasedDate
        elif self.index == "updated": index = 3 #LastUpdated
        index = f"&sortField={index}"
      if self.versions != "none": versions = f"&gameVersion={self.versions}"
      return categories + versions + index
  def SEARCH(self, search_name, software):
    project = {}
    LOG(f'\nSearching for the related of {search_name} ...\n')
    facetsInURL = self.FACETS(software)
    if software == "Modrinth":
      # Get syntax and get data
      rJSON = GET(f'https://api.modrinth.com/v2/search?query={search_name}{facetsInURL}').json()
      # Get the list of all relevant project
      for hit in rJSON['hits']:
        # Auditing if it for server or not.
        if hit['server_side'] == 'optional' or hit['server_side'] == 'required':
          # Get a full list title : description
          print(hit['slug'], " : ", hit['description'])
          project[hit['slug']] = hit['project_id']
    elif software == 'Curseforge':
      LOG(f"Curseforge doesn't have the exact searching key for cilent-side or server-side => you may get errors when running this {self.proj_types}")
      # Because I haven't found any corresponded of project types in the search engine of Curseforge, I don't use it for searching.
      # The gameid of Minecraft is 432
      rJSON = GET(f"https://api.curse.tools/v1/cf/mods/search?gameId=432&searchFilter={search_name}{facetsInURL}").json()
      # Get the list of all relevant project
      for hit in rJSON["data"]:
        # Get a full list name: summary
        print(hit["name"], ' : ', hit["summary"])
        project[hit['name']] = str(hit["id"])
    # Checking whether your search name wrong or not. If yes => Get the name of project => Get project id
    project_names =''
    if project == {}: ERROR(f"\nSomething went wrong. Please check your search name.")
    else:
      LOG('\nType the project_name you want to download')
      project_names= input('Project_name: ')
      while project_names not in project:
        LOG('\nWrong project_names please type aigain. If you want to quit, type "None".')
        project_names= input('\nProject_name: ')
        if project_names == 'None': ERROR('Stopping...')
    return [project, project_names]
  def checklist(self, B):
    A = ['ls:', 'cannot', 'access', "'server':", 'No', 'such', 'file', 'or', 'directory']
    n = len(A)
    return any(A == B[i:i + n] for i in range(len(B)-n + 1))
  def MODPACK(self, file_name, software):
    # settings up
    sleep(40)
    %cd $drive_path
    server_name = self.server_name
    path_drive = file_name
    while path_drive.find('.') != -1: path_drive = path_drive[path_drive.find('.')+1:]
    path_drive = file_name[: file_name.find('.' + path_drive)]
    if exists(f'{drive_path}/{self.server_name}/tmp/{path_drive}') == False:
      MKDIR(f'{self.server_name}/tmp/{path_drive}')
      sleep(40)
    # Unzipping modpack
    ! unzip -q '{server_name}/tmp/{file_name}' -d '{server_name}/tmp/{path_drive}' > /dev/null &&echo 'Unzip done' || echo 'Failed to unzip'
    sleep(30)
    # Copy the directory in orverrides and paste it (overrides includes the config files of the developer)
    listing_file = !ls '{server_name}/tmp/{path_drive}/overrides'
    if self.checklist(listing_file): ERROR('This is not a modpacks')
    else:
      for fln in str(listing_file[0]).split():
        ! mv -f '{server_name}/tmp/{path_drive}/overrides/{fln}' '{drive_path}/{server_name}' > /dev/null && echo 'Moving done' || echo 'Failed to move'
        sleep(40)
    # Each page give the difference file json. The file json give full details.
    # (Modrinth: modrinth.index.json- Download link, Curseforge: manifest.json - fileID, projectID) for mods which is included in modpack.
    if software == 'Modrinth':
      with ZipFile(f'{server_name}/tmp/{file_name}') as myzip:
        manifest = loads(myzip.read('modrinth.index.json'))['files']
        for file_ in manifest:
          path_ = file_["path"].split("/")[0]; file_name_ = file_["path"].split("/")[1]
          DOWNLOAD_FILE(url = file_["downloads"][0], path = f'{drive_path}/{self.server_name}/{path_}', file_name = file_name_)
    else:
      with ZipFile(f'{server_name}/tmp/{file_name}') as myzip:
        manifest = loads(myzip.read('manifest.json'))['files']
        for file_ in manifest:
          project_id =  file_["projectID"]; file_ID = file_["fileID"]; rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}/files/{file_ID}').json()["data"]
          DOWNLOAD_FILE(url = file_["downloads"], path = f'{drive_path}/{self.server_name}/mods', file_name = rJSON["displayName"])
  def Install_(self, search_name, software):
    LOG(f'Acessing: {self.server_name}')
    if self.choice == 'url':
      url = self.url
      # Find file_name in download url
      filename = input('File name (optional) : ')
      if filename == '':
        filename = url[url.find("/") + 1:]
        while filename.find("/")!= -1: filename = filename[filename.find("/") + 1:]
      else:
        format = input('Format : ')
        if f".{format}" not in filename: filename += f".{format}"
      # Download file
      if ' ' in filename: filename = filename.replace(' ', '_')
      DOWNLOAD_FILE(url= url, path = self.path, file_name= filename)
      if self.proj_types == 'tmp': self.MODPACK(file_name= filename, software= software); sleep(40)
      LOG('\nCompleted')
    elif self.choice == 'upload_file':
      uploaded = fls.upload()
      try:
        file_ = [fn for fn in uploaded.keys()][0] # Get the name of the uploaded file
        filename = input('File name (optinal)') # The file name which you want to be
        if filename != '':
          format = input('Format : ')
          if format != '' and f".{format}" not in filename: filename += f".{format}"
          else: format = file_[file_.find('.'):]
          try:
            ! sudo mv -f '{drive_path}/{file_}' '{self.path}/{filename}.{format}'
          except: ERROR("Lol, you didn't upload file yet.")
        else:
          ! sudo mv -f '{drive_path}/{file_}' '{self.path}/{file_}'
        if self.proj_types == 'tmp': self.MODPACK(file_name= filename, software= software); sleep(40)
        LOG('\nCompleted.')
      except: ERROR("Lol, you didn't upload file yet.")
    elif self.choice == 'search':
      a = self.SEARCH(search_name, software)
      project = a[0]; project_names = a[1]; check = False;
      project_id = project[project_names]
      if software == 'Modrinth': rJSON = GET(f'https://api.modrinth.com/v2/project/{project_id}/version').json()
      else: rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}').json()['data']['latestFilesIndexes'];
      for data in rJSON:
        if software == 'Curseforge': gameversions= data["gameVersion"]
        else: gameversions = data["game_versions"]
        if self.versions in gameversions:
          if software == 'Curseforge': files = data['filename']; url = f'https://www.curseforge.com/api/v1/mods/{project_id}/files/' + str(data['fileId']) + '/download'
          else: files = data['files'][0]['filename']; url = data['files'][0]['url']
          if ' ' in files: files = files.replace(' ', '_')
          DOWNLOAD_FILE(url= url, path = self.path, file_name= files)
          check = True
        if check == True and self.proj_types == 'tmp':  self.MODPACK(file_name= files, software= software); LOG('\nCompleted.'); break
        elif check: LOG('\nCompleted.'); break
      if check == False: ERROR(f"It seems that {self.software} doesn't support this {self.proj_types}")
    elif choice == 'install_geysermc':
      if self.categories != 'none' and self.colabconfig['tunnel_service'] == 'playit' and self.categories != 'forge':
        if 'velocity' in self.categories:
          rJSON = GET('https://api.papermc.io/v2/projects/velocity').json()['versions']
          if self.versions not in rJSON: ERROR('Not found versions')
          if exists(f'{drive_path}/{server_name}/plugins') == False:
            MKDIR(f'{drive_path}/{server_name}/plugins')
            sleep(40)
          plugin = ['ViaVersion', 'ViaBackwards']
          for plugins in plugin:
            rJSON = GET(f'https://hangar.papermc.io/api/v1/projects/{plugins}/versions').json()['result']; check = False
            if self.categories.upper() in rJSON[0]['platformDependenciesFormatted']:
              for hit in rJSON:
                if self.versions in hit['platformDependencies'][self.categories.upper()]:
                  DOWNLOAD_FILE(url = hit["downloads"][self.categories.upper()]["downloadUrl"], path = f'{drive_path}/{self.server_name}/plugins', file_name = hit['downloads'][self.categories.upper()]["fileInfo"]['name'])
                  check = True; break
              if check == False: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
              else: LOG(f'Installing {plugins} done. Try to install  for more supporter version')
            else: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/geyser/versions/latest/builds/latest/downloads/velocity', path = f'{drive_path}/{server_name}/plugins', file_name = 'Geyser-Velocity.jar')
          LOG('You only need to install Floodgate on the BungeeCord or Velocity proxy server, unless you want to use the Floodgate API on the backend servers. Additionally, it will display Bedrock edition skins properly.')
          LOG('For more details. Check out https://wiki.geysermc.org/floodgate/setup/')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/floodgate/versions/latest/builds/latest/downloads/velocity', path = f'{drive_path}/{server_name}/plugins', file_name = 'floodgate-velocity.jar')
        elif 'fabric' in self.categories:
          rJSON = GET('https://meta.fabricmc.net/v2/versions/game').json()['version']
          # warningserver_version
          if self.versions < rJSON and self.versions >= '1.17':
            LOG('Geyser-Fabric run only on 1.21.')
            LOG('To use Geyser with an older server version, you can use Geyser on a BungeeCord/Velocity proxy or Geyser-paper instead')
          else: ERROR(f"Your server_type isn't compatible for running Geyser_MC")
          LOG('Geyser only works with server-side mods. Mods that require a client-side install will not work!')
          LOG('Download geyser mc mods')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/geyser/versions/latest/builds/latest/downloads/fabric', path = f'{drive_path}/{self.server_name}/mods', file_name= 'Geyser-Fabric.jar')
          # Geyser mc mod require fabric api.
          LOG('Download Fabric api (requirement)')
          rJSON = GET(f'https://api.modrinth.com/v2/project/P7dR8mSH/version').json()
          check = False
          for data in rJSON:
            if self.versions in data["game_versions"]:
              files = data['files'][0]; url = files['url']
              DOWNLOAD_FILE(url= url, path = self.path, file_name= files['filename']); check = True
            if check: break
          if check == False: ERROR(f"It seems that theresn't fabric api for this minecraft version")
          LOG('Download floodgate')
          rJSON = GET(f'https://api.modrinth.com/v2/project/bWrNNfkb/version').json()
          check = False
          for data in rJSON:
            if versions in data["game_versions"]:
              files = data['files'][0]; url = files['url']
              DOWNLOAD_FILE(url= url, path = self.path, file_name= files['filename']); check = True
            if check: break
          if check == False: ERROR(f"It seems that theresn't fabric api for this minecraft version")
        elif 'paper' in self.categories or 'purpur' in self.categories:
          rJSON = GET('https://api.papermc.io/v2/projects/paper').json()['versions'][-1]
          if self.versions < rJSON and self.versions >= '1.17':
            plugin = ['ViaVersion', 'ViaBackwards']
            for plugins in plugin:
              rJSON = GET(f'https://hangar.papermc.io/api/v1/projects/{plugins}/versions').json()['result']; check = False
              if self.categories.upper() in rJSON[0]['platformDependenciesFormatted']:
                for hit in rJSON:
                  if self.versions in hit['platformDependencies'][self.categories.upper()]:
                    DOWNLOAD_FILE(url = hit["downloads"][self.categories.upper()]["downloadUrl"], path = f'{drive_path}/{self.server_name}/plugins', file_name = hit['downloads'][self.categories.upper()]["fileInfo"]['name'])
                    check = True; break
                if check == False: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
                else: LOG(f'Installing {plugins} done. Try to install  for more supporter version')
              else: LOG(f"{plugins} can't be downloaded in your velocity server. You can try to install and upload it through website")
          else: ERROR(f"Your server_type isn't compatible for running Geyser_MC")
          LOG('Download geyser mc plugin')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/geyser/versions/latest/builds/latest/downloads/spigot', path = f'{drive_path}/{self.server_name}/plugins', file_name= 'Geyser-Spigot.jar')
          LOG('Download floodgate')
          DOWNLOAD_FILE(url = 'https://download.geysermc.org/v2/projects/floodgate/versions/latest/builds/latest/downloads/spigot', path = f'{drive_path}/{self.server_name}/plugins', file_name = 'floodgate-spigot.jar')
        # Set up notification
        self.colabconfig['Geysermc']= 'notdone'
        dump(self.colabconfig, open(COLABCONFIG(self.server_name),'w'))
        # Ping
        LOG('\nInstalling done. Try to rerun the server to configuration.')
      else: ERROR(f"Your server_type isn't compatible for running Geyser_MC")
    elif choice == 'dynmap support':
      if self.categories == 'paper' or self.categories == 'purpur' or self.categories == 'fabric':
        if self.colabconfig['tunnel_service'] != 'ngrok' or self.colabconfig['tunnel_service'] != 'playit' or self.colabconfig['tunnel_service'] = 'localtonet' or self.colabconfig['tunnel_service'] =='zrok': ERROR('The server is not compatible to use dynmap')
        if self.categories == 'fabric': software = 'Curseforge'
        else: software = 'Modrinth'
        check = False
        if software == 'Modrinth': project_id = 'fRQREgAc'; rJSON = GET(f'https://api.modrinth.com/v2/project/{project_id}/version').json()
        else: project_id = '59433'; rJSON = GET(f'https://api.curse.tools/v1/cf/mods/{project_id}').json()['data']['latestFilesIndexes'];
        for data in rJSON:
          if software == 'Curseforge': gameversions= data["gameVersion"]
          else: gameversions = data["game_versions"]
          if self.versions in gameversions:
            if software == 'Curseforge': url = f'https://www.curseforge.com/api/v1/mods/{project_id}/files/' + str(data['fileId']) + '/download'
            else: url = data['files'][0]['url']
            DOWNLOAD_FILE(url= url, path = self.path, file_name= 'dynmap-server.jar')
            check = True
          if check: LOG('\nCheck https://github.com/webbukkit/dynmap/wiki/Installation-Setup-of-Dynmap-on-Linux for more informations.'); LOG('\nWe have done the step 1 in the set up dynmap guide. Enjoy!'); break
        if check == False: ERROR(f"It seems that there is an error in installing dynmap plugin/mod")
      else: ERROR('Wrong choice')

---

# **📁 File Management**
---


In [ ]:
from os.path import exists
from time import sleep

# @markdown #### Back up server or file?
server_name= SERVER_IN_USE(server_name= '')
path = '/content/drive/MyDrive/minecraft' # Default path. Change to any location you wanna
choice = 'server' # @param ['server', 'file']
file_name = ''; file_backup = '';  server_backup = ''
if choice == 'server':
  server_backup = input('Server back up name: ')
else: file_name = input('File name'); file_backup = input('File back up name: ')

# Settings path
if file_name != '':
  path1 = path + f'/{server_name}/' + file_name
  if server_backup != '':
    if exists(f'{drive_path}/{server_backup}') == False:
      !mkdir '{drive_path}/{server_backup}'
      sleep(40)
  if file_backup != '' and server_backup != '': path2 = path + f'/{server_backup}/' + file_backup
  elif file_backup != '' and server_backup == '': path2 = path + f'/{server_name}/' + file_backup
  elif file_backup == '' and server_backup != '':  path2 = path + f'/{server_backup}/' + '(back-up)'
  else: path2 = path + f'/{server_name}/' + '(back-up)'
else:
  path1 = path + f'/{server_name}'
  if server_backup != '': path2 = path + f'/{server_backup}'
  else: path2 = path + f'/{server_name}' + '(back-up)'
# Checking and zipping
if exists(path1) == False: ERROR(' Creating your minecraft server before backing up files')
if exists(path2) == True: ERROR(' Back up path exists')
# Zipping
!zip -r '{path2}.zip' '{path1}' && echo "Zipping done!" || echo "Zipping faled."
# Download
choice = input('Download? (y/n) : ')
if choice == 'y': fls.download(f'{path2}.zip')

In [ ]:
from google.colab import files as fls

# @markdown #### World map uploader
choice = 'url' # @param ['upload_file', 'url']
world = 'see all available' # @param ['see all available', 'world', 'world_nether', 'world_the_end']
server_name = SERVER_IN_USE(server_name)

if exists(f"{drive_path}/{server_name}/server.properties") == False: ERROR(' Running your minecraft server before editing properties')
else:
  LOG("This only applied on .zip files. Others will work too  but won't be supported")
  if world == 'see all available':
    a = !ls Minecraft-server | grep 'world'
    LOG('All world file found: ')
    for world in str(a[0]).split(): print(world)
    world = input('World file: ')
  if choice == 'url':
    # Download file
    url = input('Download link : ')
    DOWNLOAD_FILE(url= url, path= f"{drive_path}/{server_name}", file_name= 'tmp.zip')
    # Unzipping
    !sudo unzip tmp.zip -d $world >/dev/null
  else:
    %cd $drive_path/$server_name
    # Download file
    uploaded = fls.upload()
    try:
      file_name = [fn for fn in uploaded.keys()][0] # Get the name of the uploaded file
      # Unzipping
      !sudo unzip $file_name -d $world > /dev/null && echo "Unzipped successfully" || echo "Failed to unzip"

    except: ERROR("Lol, you didn't upload file yet.")
  %cd $drive_path